# 분류 (classification)

+ 머신러닝에서 회귀(regression)과 함께 가장 기본적인 데이타 분석 방법
+ 분류는 데이타가 어느 범주(category)에 해당하는지 판단하는 것이다.


## [1]이항 분류

+ 정답의 범주가 두 개인 분류 문제
+ ex) 와인의 상태에 따라 레드와인인지 화이트 와인인지 구별

               
<table>
<tr><th>속성</th><th>설명</th></tr>
<tr><td>fixed acidity</td><td>주석산(Tartaric acid)</td></tr>
<tr><td>volatile acidity</td><td>초산(Acetic acid)</td></tr>
<tr><td>citric acid</td><td>구연산</td></tr>
<tr><td>residual sugar</td><td>당도</td></tr>
<tr><td>chlorides</td><td>영화물(소금)</td></tr>
<tr><td>free sulfur dioxide</td><td>자유 이산화황(산화방지제, 방부제 역할)</td></tr>
<tr><td>total sulfur dioxide</td><td>총 이산화황</td></tr>
<tr><td>density</td><td>밀도</td></tr>
<tr><td>pH</td><td>산도</td></tr>
<tr><td>sulphates</td><td>황산칼륨</td></tr>
<tr><td>alcohol</td><td>알코올 도수</td></tr>
<tr><td>quality</td><td>품질(0~10)</td></tr>
<tr><td>type</td><td>종류(레드:1 , 화이트:0)</td></tr>
</table>

In [ ]:
# (1-1) 와인 데이터셋 불러오기
import pandas as pd
wine = pd.read_csv('./dataset/wine.csv', header=None)
wine.columns = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide',
               'total sulfur dioxide','density','pH','sulphates','alcohol','quality',
               'type']
print(wine.shape)
wine.tail()

In [ ]:
# [확인] 레드 와인과 화이트 와인 데이타 비율 확인

# type 속성의 값을 수로 확인
print(wine['type'].value_counts())

# type 히스토그램
import matplotlib.pyplot as plt
plt.hist(wine['type'])
plt.xticks([0, 1])
plt.show()

In [ ]:
# [확인] 데이터 요약 정보 확인
#     외부에서 불러온 데이터의 경우 어떤 값으로 구성되어 있는지 확인하고 데이타를 섞고 정규화를 해야 한다.
#     정규화 과정에서 데이터에 숫자가 아닌 값이 들어가며 에러을 유발한다.
#     모두 숫자형에 non-null 로 null가 존재하지 않음을 확인하다.

print(wine.info())
wine.head()


In [ ]:
# (1-2) 데이터 정규화

wine_norm = (wine - wine.min()) / (wine.max() - wine.min())

print(wine_norm.describe())
# 결과를 확인하면 min은 0이고 max는 1로 모두 0~1 사이의 데이타로 정규화가 된 것이다.

wine_norm.head()



In [ ]:
# (1-3) 데이터 섞은 후 numpy array로 변환
#       df.sample(frac) : 전체 데이타프레임에서 frac 인수로 지정된 비율만큼의 행을 랜덤하게 뽑아서 새로운 데이타프레임을 만듬
#       frac=1 : 100% 즉 모든 데이터를 뽑아서 섞는 것과 동일한 효과이다.

import numpy as np
wine_shuffle = wine_norm.sample(frac=1)
display(wine_shuffle.head())

wine_np = wine_shuffle.to_numpy()
display(wine_np[:5])

# 첫번째 출력 결과에서  type을 보면 원래는 레드와인이 앞에 있어서 0.0만 나와야하지만 섞여서 나온다.

In [ ]:
# (1-4) train 데이터와 test 데이터로 분리
#         훈련 데이타 80% + 테스트 데이타 20%
#         기존 12개의 속성은 X가 되고 마지막에 새롭게 추가한 속성은 Y가 된다.
import tensorflow as tf
train_idx = int(len(wine_np) * 0.8)
train_X, train_Y = wine_np[:train_idx, :-1], wine_np[:train_idx, -1]
test_X, test_Y = wine_np[train_idx:, :-1], wine_np[train_idx:, -1]
print(train_X[0])
print(train_Y[0])
print(test_X[0])
print(test_Y[0])

# tf.keras.utils.to_categorical : 정답 행렬을 one-hot encoding 방식으로 바꾼다
# one-hot encoding : 정답에 해당하는 인덱스의 값에는 1을 넣고, 나머지 인덱스는 모두 0으로 넣는 방식
# num_classes : 정답 클래스의 갯수
# 정답이 레드 와인(0.0)이라면 [1. 0.]이 되고 화이트 와인(1.0)이라면 [0. 1.]이 된다.
train_Y = tf.keras.utils.to_categorical(train_Y, num_classes=2)
test_Y = tf.keras.utils.to_categorical(test_Y, num_classes=2)
print(train_Y[0])
print(test_Y[0])

In [ ]:
# (2) 와인 데이터셋 분류 모델 생성하고 실행
#     회귀 모델과 비슷해 보이지만 마지막 레이어의 활성화 함수가 softmax이다.
#     마지막 레이어의 뉴런의 갯수가 2개인 것은 원-핫 인코딩으로 변환한 정답의 차원 수와 동일하다.
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=48, activation='relu', input_shape=(12,)),
    tf.keras.layers.Dense(units=24, activation='relu'),
    tf.keras.layers.Dense(units=12, activation='relu'),
    tf.keras.layers.Dense(units=2, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.07), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# loss에 주로 사용하던 mse 대신 categorical_crossentropy 지정
# metrics=['accuracy'] : 분류 문제는 정확도(accuracy)로 퍼포먼스를 측정하기 때문에 정확도를 설정하는 것이 필수이다.

[참고] [ 크로스 엔트로피(cross entropy) ](https://wordbe.tistory.com/entry/ML-Cross-entropyCategorical-Binary%EC%9D%98-%EC%9D%B4%ED%95%B4)

[참고] [ softmax 간단한 이해 ](https://m.blog.naver.com/wideeyed/221021710286)

In [ ]:
# (3) 와인 데이터셋 분류 모델 학습
history = model.fit(train_X, train_Y, epochs=25, batch_size=32, validation_split=0.25)

[결과] 정확도가 100% 가까운 매우 좋은 결과를 유지하고 있다

Epoch 25/25
3897/3897 [==============================] - 0s 73us/sample - loss: 0.0339 - accuracy: 0.9905 - val_loss: 0.0443 - val_accuracy: 0.9885

    훈련 손실값 (loss)
    훈련 정확도 (acc)
    검증 손실값 (val_loss)
    검증 정확도 (val_acc)

In [ ]:
# [확인] 분류 모델 학습 결과 시각화
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0.7, 1)
plt.legend()

plt.show()

In [ ]:
# (4) 분류 모델 평가
model.evaluate(test_X, test_Y)

# 정확도(accuracy)가 0.9931로 거의 오차 없이 레드와인와 화이트와인을 분류했다

In [ ]:
# (5) 주어진 자료로 레드와인인지 화이트 와인인지 감별
# 정답이 레드 와인(0.0)이라면 [1. 0.]이 되고 화이트 와인(1.0)이라면 [0. 1.]이 된다.

x_values = ([0.30578512, 0.29333333, 0.09638554, 0.01993865, 0.12624585, 0.03819444,
 0.06682028, 0.18681319, 0.51162791, 0.2247191,  0.2173913,  0.66666667],)
y_pred = model.predict(x_values)
y_pred  # 화이트와인